In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/DNA_classifier/research'

In [2]:
os.chdir("../")
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/DNA_classifier'

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    dwn_data_file: Path
    STATUS_FILE: Path
    all_schema: dict

In [18]:
from dnaseq.constants import *
from dnaseq.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            dwn_data_file=config.dwn_data_file,
            STATUS_FILE=config.STATUS_FILE,
            all_schema=schema
        )
        
        return data_validation_config

In [27]:
import pandas as pd

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_table(self.config.dwn_data_file)
            all_cols = list(data.columns)

            all_schema = list(self.config.all_schema.keys())
            
            for col in all_schema:
                if col not in all_cols:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                elif validation_status==None:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

In [28]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()

except Exception as e:
    raise e

[2024-03-27 21:51:41,273: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-27 21:51:41,276: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-27 21:51:41,278: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-27 21:51:41,279: INFO: common: created directory at: artifacts]
[2024-03-27 21:51:41,280: INFO: common: created directory at: artifacts/data_validation]
['sequence', 'class']
['sequence', 'class', 'kmers']
